Word2Vec + Decision Tree Regressor

In [1]:
# imports
import pandas as pd
import tensorflow as tf
import preprocess_kgptalkie as ps
import nltk

In [ ]:
# installations
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

Dataset Preprocessing

In [58]:
# training data
train_dataset = pd.read_csv('drugsComTrain_raw.csv')
train_dataset['review'].dropna(inplace=True)
train_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,19-Sep-15,79


In [18]:
# testing data
test_dataset = pd.read_csv('drugsComTest_raw.csv')
test_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4
...,...,...,...,...,...,...,...
53761,159999,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10,13-Sep-14,43
53762,140714,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9,8-Oct-16,11
53763,130945,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8,15-Nov-10,7
53764,47656,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1,28-Nov-11,20


In [59]:
# preprocessing: lowercase, punctuation removal, tokenization, stop word removal, stemming, lemmatization
train_dataset['review'] = train_dataset['review'].apply(lambda x: str(x).lower())
train_dataset['review'] = train_dataset['review'].apply(lambda x: ps.remove_special_chars(x))

In [20]:
train_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,it has no side effect i take it in combination...,9,20-May-12,27
1,95260,Guanfacine,ADHD,my son is halfway through his fourth week of i...,8,27-Apr-10,192
2,92703,Lybrel,Birth Control,i used to take another oral contraceptive whic...,5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,this is my first time using any form of birth ...,8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,suboxone has completely turned my life around ...,9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,i wrote my first report in midoctober of 2014 ...,10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,i was given this in iv before surgey i immedia...,1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,limited improvement after 4 months developed b...,2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,i039ve been on thyroid medication 49 years i s...,10,19-Sep-15,79


In [61]:
# tokenization
train_dataset['review'] = train_dataset['review'].astype(str)

nltk.download('punkt')

def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]   

train_dataset['review'] = train_dataset.apply(lambda x: tokenize(x['review']), axis=1)

# sent_tokenize(train_dataset)
# train_data['review'] = df.apply(lambda x: nltk.word_tokenize(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [62]:
train_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"[it, has, no, side, effect, i, take, it, in, c...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"[my, son, is, halfway, through, his, fourth, w...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"[i, used, to, take, another, oral, contracepti...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"[this, is, my, first, time, using, any, form, ...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"[suboxone, has, completely, turned, my, life, ...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"[i, wrote, my, first, report, in, midoctober, ...",10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,"[i, was, given, this, in, iv, before, surgey, ...",1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,"[limited, improvement, after, months, develope...",2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"[been, on, thyroid, medication, years, i, spen...",10,19-Sep-15,79


In [63]:
# stop word removal
from nltk.corpus import stopwords
stopwords = list(stopwords.words('english'))

train_dataset['review'] = [word for word in train_dataset['review'] if word not in stopwords]

In [64]:
train_dataset.head(3)

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"[it, has, no, side, effect, i, take, it, in, c...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"[my, son, is, halfway, through, his, fourth, w...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"[i, used, to, take, another, oral, contracepti...",5,14-Dec-09,17


AttributeError: 'WordListCorpusReader' object has no attribute 'word'